## 파일 가져오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## train, valid, test set 생성

In [81]:
path = '/content/drive/MyDrive/2학년 1학기/DACON/hand_gesture_data/'

df = pd.read_csv(path + 'train.csv').drop('id', axis = 1)
df_test = pd.read_csv(path + 'test.csv').drop('id', axis = 1)

features = df.iloc[:, :32]
labels = df.iloc[:, 32]

train_x, valid_x, train_y, valid_y = train_test_split(features, labels, test_size = 0.2, 
                                                    random_state = 908, stratify = labels)

print(len(train_x), len(valid_x), len(train_y), len(valid_y))

1868 467 1868 467


In [82]:
train_x['target'] = train_y
valid_x['target'] = valid_y

train_data = torch.tensor(train_x.to_numpy()).float()
valid_data = torch.tensor(valid_x.to_numpy()).float()
test_data = torch.tensor(df_test.to_numpy()).float()

train_data

tensor([[-26.0182,  -7.0139,  -3.1624,  ...,   7.8912,  -0.9049,   2.0000],
        [  4.9438,   6.9123, -30.0757,  ...,  -9.8902,  -5.0376,   1.0000],
        [  1.9227,   3.0482,   9.0198,  ...,  15.9428,   9.0623,   3.0000],
        ...,
        [ -1.9991,  -3.8975,   1.0088,  ..., -14.9780,  -5.0239,   3.0000],
        [ -3.0107,  -0.9051,   4.0431,  ..., -12.9826,   0.8498,   1.0000],
        [  2.1036,   4.0618, -34.9921,  ...,  44.9914,  -7.8544,   1.0000]])

In [83]:
valid_data

tensor([[-9.0225e+00,  9.7237e-01,  7.0467e+00,  ..., -3.2061e+01,
         -1.1030e+01,  2.0000e+00],
        [-3.0089e+00,  3.0868e+00, -4.5939e+01,  ...,  8.0887e+01,
          1.4929e+01,  3.0000e+00],
        [ 1.3871e-01, -2.1820e+00,  2.6881e+01,  ..., -3.0998e+01,
         -5.9963e+00,  1.0000e+00],
        ...,
        [-8.9184e+00,  3.1382e+00,  2.8129e+01,  ..., -1.0817e+01,
          1.0706e+00,  3.0000e+00],
        [-1.9682e+00,  5.8605e-02, -8.9131e+00,  ..., -9.0255e+00,
         -3.0565e+00,  1.0000e+00],
        [ 3.8140e+00, -8.0363e+00,  5.9204e+00,  ..., -4.2964e+01,
         -3.9029e+00,  0.0000e+00]])

In [84]:
test_data

tensor([[ 6.7135e-02,  5.0404e+00, -2.9654e+00,  ...,  4.0444e+00,
         -1.9933e+00,  1.0856e+01],
        [ 6.8907e+00, -1.2011e+01,  2.4882e+01,  ..., -2.8528e+00,
          5.6028e+01,  5.2967e+01],
        [-4.8097e+00,  9.6286e-01, -1.8936e+00,  ..., -6.9175e+00,
          2.8424e+00, -3.0814e+00],
        ...,
        [ 4.0359e+00, -1.0138e+01,  5.9432e+00,  ...,  1.9217e+00,
          1.3952e+01,  1.1866e+01],
        [ 8.9205e+00,  3.3165e-02,  2.0058e+01,  ...,  9.9685e+00,
         -3.0971e+00,  7.9662e+00],
        [ 1.0422e+00, -2.9284e+00, -3.4982e+01,  ...,  3.8116e+00,
          7.7962e+01, -2.2974e+01]])

In [85]:
def total_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size)

    return train_loader, valid_loader, test_loader

## Simple DNN 생성, 학습

In [86]:
class categorical(nn.Module):
    def __init__(self):
        super(categorical, self).__init__()

        self.fc1 = nn.Linear(32, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 4)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim = 1)


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        output = self.softmax(self.fc3(x))
       
        return output

In [87]:
def train(model, train_loader, valid_loader, optimizer, criterion, epochs, early_stopping_counter):
    model.train()
    min_val_loss = 999999
    counter = 0

    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        train_loss = []
        valid_loss = []

        for batch_data in train_loader:
            features, labels = batch_data[:, :32], batch_data[:, 32].long()

            optimizer.zero_grad()
            output = model(features)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            epoch_loss += (loss.item() / len(train_loader))
            train_loss.append(loss.item())

        #---------------------------------------------------------------------#

        model.eval()

        with torch.no_grad():
            for batch_data in valid_loader:
                features, labels = batch_data[:, :32], batch_data[:, 32].long()

                output = model(features)
                loss = criterion(output, labels)

                valid_loss.append(loss.item())

        #---------------------------------------------------------------------#

        print('Epoch [{}/{}] / Train_Loss : {} / Valid_Loss : {}'.format(epoch, epochs, np.average(train_loss), np.average(valid_loss)), end = ' ')

        
        if min_val_loss > np.average(valid_loss):
            torch.save(model, path + 'hand_dnn.pt')
            min_val_loss = np.average(valid_loss)
            print('/ New Model Saved.')
            counter = 0
        else:
            counter += 1
            if counter == early_stopping_counter:
                print('/ Early Stopping.')
                break
            else:
                print('/ Counter [{}/{}]'.format(counter, early_stopping_counter))

In [88]:
batch_size = 16
epochs = 150
learning_rate = 0.01
early_stopping_counter = 20

model = categorical()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

train_loader, valid_loader, test_loader = total_loader(batch_size)

train(model, train_loader, valid_loader, optimizer, criterion, epochs, early_stopping_counter)

Epoch [1/150] / Train_Loss : 1.3898706670500274 / Valid_Loss : 1.3527858217557271 / New Model Saved.
Epoch [2/150] / Train_Loss : 1.2585030763577192 / Valid_Loss : 1.2227944691975912 / New Model Saved.
Epoch [3/150] / Train_Loss : 1.2025370888220959 / Valid_Loss : 1.1816839555899301 / New Model Saved.
Epoch [4/150] / Train_Loss : 1.1591277428162403 / Valid_Loss : 1.1602157910664876 / New Model Saved.
Epoch [5/150] / Train_Loss : 1.1377957758740482 / Valid_Loss : 1.1637830038865407 / Counter [1/20]
Epoch [6/150] / Train_Loss : 1.1351570359662049 / Valid_Loss : 1.1297813634077707 / New Model Saved.
Epoch [7/150] / Train_Loss : 1.1404721609547608 / Valid_Loss : 1.2099351723988852 / Counter [1/20]
Epoch [8/150] / Train_Loss : 1.1159179373684092 / Valid_Loss : 1.1881167511145274 / Counter [2/20]
Epoch [9/150] / Train_Loss : 1.1122466509158795 / Valid_Loss : 1.14734574953715 / Counter [3/20]
Epoch [10/150] / Train_Loss : 1.0929047526457372 / Valid_Loss : 1.1698720335960389 / Counter [4/20]
E

In [89]:
model = torch.load(path + 'hand_dnn.pt')
model.eval()

with torch.no_grad():
    test_labels = []

    for batch_data in test_loader:
        output = model(batch_data)

        for i in range(len(output)):
            test_labels.append(int(np.argmax(output[i])))
                    
test_labels[:10]

[0, 0, 0, 3, 2, 0, 0, 1, 1, 1]

## 예측 결과 생성

In [90]:
df_test = pd.read_csv(path + 'test.csv')
df_test['target'] = test_labels
df_final = df_test[['id', 'target']]
df_final

,id,target
0,1,0
1,2,0
2,3,0
3,4,3
4,5,2
...,...,...
9338,9339,0
9339,9340,1
9340,9341,0
9341,9342,0


In [91]:
df_final.to_csv(path + 'DNN_earlystopping_submission.csv', index = False)